# Probability

In [1]:
%matplotlib inline
from collections import Counter, OrderedDict, defaultdict
from functools import reduce, partial
from itertools import count, combinations
from math import sqrt, pi, sin, cos, atan, radians, degrees
from random import random, randint, choice
from time import time
import json
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image, ImageDraw, ImageColor
import seaborn as sns

from drawing import VectorImage, Turtle, show_image

-----

# A) Girl/Boy Proportion Problem

- All parents have a new child until a boy is born.
- A new child is boy or girl with equal probability.
- Start with the 50/50 proportion.
- What will be the proportion of boys?

** Approach 1: intuiton. ** 
50 % (probability of a new child is always 50 %)

** Approach 2: reasoning (deduction). ** 
todo

** Approach 3: simulation. **

In [ ]:
def compute_proportion(n_people=100, n_iter=100, p_boy=0.5):
    # 1 = boy, 0 = girl
    population = [randint(0, 1) for _ in range(n_people)]
    for i_iter in range(n_iter):
        population = generate_new_population(population, p_boy)
    return sum(population) / len(population)

def generate_new_population(population, p_boy):
    new_population = []
    


compute_proportion()

TODO: modification to the rules, what happens?

-----

# B) Nonrandom Numbers

TODO:
- ...

In [14]:
for i in range(1, 8):    
    with open('random/random{0}.txt'.format(i)) as f:
        numbers = list(map(int, f.read().split()))
        print('random' + str(i) + ':')
        # Analysis of frequences of numbers
        print(Counter(numbers))
        # Analysis of bigrams
        bigrams = [
            str(numbers[i]) + str(numbers[i+1])
            for i in range(len(numbers)-1)]
        print('distinct birgrams', len(set(bigrams)))
        #print(Counter(bigrams))
        print()
        
# Probably not random:
# 1 ...
# 2 (1-grams: 1244x number 5)
# 4 (2-grams: only 18 from 36 possible pairs)
# TODO: frequency in every 2nd/3rd numbers
# TODO: Matrix of bigrams
# TODO: nejldelsi posloupnost, ktera se opakuje N-krat (2-krat?)
# TODO: ML approach - can we predict the last 30% of numbers

random1:
Counter({4: 834, 2: 834, 5: 833, 3: 833, 1: 833, 6: 833})
distinct birgrams 36

random2:
Counter({5: 1244, 1: 856, 4: 845, 3: 829, 6: 798, 2: 428})
distinct birgrams 36

random3:
Counter({6: 850, 3: 850, 2: 836, 4: 824, 5: 821, 1: 819})
distinct birgrams 36

random4:
Counter({1: 852, 4: 850, 3: 829, 2: 827, 6: 823, 5: 819})
distinct birgrams 18

random5:
Counter({6: 859, 4: 843, 5: 842, 2: 836, 3: 812, 1: 808})
distinct birgrams 36

random6:
Counter({2: 851, 3: 849, 1: 849, 4: 846, 5: 803, 6: 802})
distinct birgrams 36

random7:
Counter({2: 842, 1: 838, 4: 834, 5: 833, 6: 830, 3: 823})
distinct birgrams 36



In [15]:
6 * 999 / (6 * 999  + 9940)

0.3761767290071545

-----

# D) Bayes Theorem

Intuitive guesses for dice being fair:
- 10 dice, 5 rols: 0.3
- 100 dice, 3 rolls: 0.9 (should be 0.3)
- 1000 dice, 5 rolls: 0.5

Post comment: my intutition was absolutely wrong!

In [28]:
# First, using Bayes Theorem:

def compute_fair_probability(n_dice=100, n_rolls=3, n_faces=6):
    """Return probabilty of a dice taken randomly from a set
       of n_dice being fake, given that only one is fake,
       and given that we observe n_rolls of 6 in a row.
    """
    fake_prior = 1 / n_dice
    fake_conditional = 1
    fake_likelihood = fake_prior * fake_conditional
    
    fair_prior = 1 - fake_prior
    fair_conditional = 1 / (n_faces ** n_rolls)
    fair_likelihood = fair_prior * fair_conditional
    
    observation_probability = fake_likelihood + fair_likelihood
    fair_probability = fair_likelihood / observation_probability
    return fair_probability
    
settings = [(2, 1), (10, 5), (100, 3), (1000, 5)]
for n_dice, n_rolls in settings:
    prob = compute_fair_probability(n_dice, n_rolls)
    print('n={0} x={1} --> {2}'.format(n_dice, n_rolls, prob))

n=2 x=1 --> 0.14285714285714285
n=10 x=5 --> 0.0011560693641618498
n=100 x=3 --> 0.3142857142857143
n=1000 x=5 --> 0.11384615384615385


In [31]:
def simulate_fair_probability(n_dice=100, n_rolls=3, n_faces=6,
                              n_iter=100000):
    """The same as above, estimated by simulation.
    
    We will use simple rejection sampling.
    """
    n_success, n_total = 0, 0
    for i_iter in range(n_iter):
        if randint(1, n_dice) == n_dice:
            # fake dice
            n_total += 1
        else:
            for i_roll in range(n_rolls):
                if randint(1, n_faces) != n_faces:
                    break
            else:  # all rolls were 6
                n_success += 1
                n_total += 1    
    return n_success / n_total
    
settings = [(2, 1), (10, 5), (100, 3), (1000, 5)]
for n_dice, n_rolls in settings:
    prob = simulate_fair_probability(n_dice, n_rolls)
    print('n={0} x={1} --> {2}'.format(n_dice, n_rolls, prob))

n=2 x=1 --> 0.14637024369388626
n=10 x=5 --> 0.0014885382554331646
n=100 x=3 --> 0.31288766368022053
n=1000 x=5 --> 0.11818181818181818
